In [4]:
# Import des bibliothèques

import os
from pathlib import Path
import numpy as np
import pandas as pd
import sys
import time

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

from tokenizers import Tokenizer
# from transformers import AutoTokenizer
from transformers import BertForMaskedLM, AutoTokenizer, AutoModelForMaskedLM
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer
from tokenizers.pre_tokenizers import Whitespace


In [2]:
# Détermination du Path

DATASET_PATH = Path("./data/text")


In [5]:
# Récupération des données textuelles

VOC_SIZE = 1000

def load_data(datapath, max_size=None):
    texts_files = list(datapath.glob("*.txt"))
    texts = []  
    for files in texts_files:
        with open(files, "r", encoding='utf8') as files:
            text = files.readlines()
            texts += text
    texts = list(set(texts))
    
    return texts

texts = load_data(DATASET_PATH)

In [7]:
# Chargement du modèle CamemBERTav2 et tokenisation du texte

model_checkpoint = "almanach/camembertav2-base"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = BertForMaskedLM.from_pretrained(model_checkpoint)

inputs = tokenizer(texts, return_tensors='pt', max_length=100
                   , truncation=True, padding='max_length')

inputs['labels'] = inputs.input_ids.detach().clone()

print(inputs.tokens(1))


You are using a model of type deberta-v2 to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForMaskedLM were not initialized from the model checkpoint at almanach/camembertav2-base and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.1.attention.self.key.bias', 'encoder.layer.1.attention.self.key.weight', 'encoder.layer.1.attention.self.quer

['[CLS]', 'Dr', '##y', 'for', '##ages', 'Ric', '##e', 'be', '##an', '(', 'Vig', '##na', 'u', '##mb', '##ella', '##ta', '),', 'ha', '##y', 'Ric', '##e', 'be', '##an', '(', 'Vig', '##na', 'u', '##mb', '##ella', '##ta', '(', 'Th', '##un', '##b', '.)', 'Oh', '##wi', '&', 'Oh', '##ashi', '),', 'ha', '##y', '\n', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [2]:
rand = torch.rand(inputs.input_ids.shape)
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [5]:
import random as rd

In [6]:
# apply the [MASK] token with the mask array 
inputs.input_ids[mask_arr] = 103

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, idx):
        self.encodings = encodings
        self.idx = idx
        self.encodings = {key: [val[i] for i in self.idx] for key, val in self.encodings.items()}
        
    def __getitem__(self, idx):
        return {key : torch.tensor(val[idx]) for key, val in self.encodings.items()}
    
    def __len__(self):
        return len(self.encodings['input_ids'])

sample_idx = [i for i in range(len(inputs.input_ids))]

shuffled_sample_idx = rd.sample(sample_idx, len(sample_idx))

train_idx = shuffled_sample_idx[:int(0.70*len(shuffled_sample_idx))]
val_idx = shuffled_sample_idx[int(0.70*len(shuffled_sample_idx)):int(0.85*len(shuffled_sample_idx))]
test_idx = shuffled_sample_idx[int(0.85*len(shuffled_sample_idx)):]
                                
dataset_train = CustomDataset(inputs, train_idx)
dataset_val = CustomDataset(inputs, val_idx)
dataset_test = CustomDataset(inputs, test_idx)

train_dataloaded = torch.utils.data.DataLoader(dataset_train, batch_size=16, shuffle=True)
val_dataloaded = torch.utils.data.DataLoader(dataset_val, batch_size=16, shuffle=True)
test_dataloaded = torch.utils.data.DataLoader(dataset_test, batch_size=16, shuffle=True)

In [5]:
#class MLM_model(nn.Module):
#    def __init__(self, model):
#        super(MLM_model, self).__init__()
#        self.history = {"epochs":[], "test":[]}
#        self.model = model
    
#    def parameters(self):
#        return self.model.parameters()

#    def forward(self, x, attention_mask, labels):
#        return self.model(x, attention_mask, labels)
    
#    def train_log(self, train_batch_losses, val_batch_losses, train_loss, validation_loss):
#        self.history["epochs"].append({"train_batch_losses":train_batch_losses, 
#                                "val_batch_losses":val_batch_losses, 
#                                "train_loss":train_loss, 
#                                "validation_loss":validation_loss})
    
#    def test_log(self, test_batch_losses, test_loss):
#        self.history["test"].append({"test_batch_losses":test_batch_losses,
#                                "test_loss":test_loss})

In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#model = MLM_model(model)
model.to(device)
print(device)

cuda


In [8]:
def train_step(module, batch, batch_idx, optimizer):
    module.train(True)
    
    inputs_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    
    outputs = module(inputs_ids, attention_mask, labels=labels)
    
    loss = outputs.loss
    print(f"\n\033[1;37mBatch loss {batch_idx+1} : {loss.item()}")
    loss.backward()
    
    torch.nn.utils.clip_grad_norm_(module.parameters(), max_norm=1.0)
    optimizer.step()
    optimizer.zero_grad()
    
    return module, loss

def eval_step(module, batch, batch_idx, optimizer=None, training=True):
    with torch.no_grad():
        
        inputs_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
    
        outputs = module(inputs_ids, attention_mask, labels=labels)
    
        loss = outputs.loss
         
        if training:
            print(f"\n\033[1;32mValidation Batch loss {batch_idx+1} : {loss.item()}")
            return module, loss
        else:
            print(f"\n\033[1;32mTest Batch loss {batch_idx+1} : {loss.item()}")
            return module, loss, outputs, labels

def train_loop(module, EPOCHS, train_dataset, val_dataset, optimizer, lr_scheduler=None):
    for epoch in range(EPOCHS):
        deb=time.time()
        
        module.train(True)
        
        train_batch_losses = []
        for batch_idx in range(len(train_dataset)):
            batch = next(iter(train_dataset))
            module, loss = train_step(module, batch, batch_idx, optimizer)
            train_batch_losses.append(loss.item())
            
        if lr_scheduler is not None:
          lr_scheduler.step()
        train_loss = np.mean(train_batch_losses)

        module.train(False)
        val_batch_losses = []
        for batch_idx in range(len(val_dataset)):
            batch = next(iter(val_dataset))
            module, loss = eval_step(module, batch, batch_idx)
            val_batch_losses.append(loss.item())
        val_loss = np.mean(val_batch_losses)

#        module.train_log(train_batch_losses, val_batch_losses, train_loss, val_loss)
        print(f"\n\033[1;33mEpoch {epoch+1} :\n\033[1;37mTraining Loss : {train_loss}")
        print(f"\033[1;32mValidation Loss : {val_loss}")
        print(f"\033[1;31mDurée epoch : {time.time()-deb} secondes")
    return module

def evaluate(module, test_dataset):
    module.train(False)
    test_batch_losses = []
    predictions = []
    true_targets = []
    for batch_idx in range(len(test_dataset)):
        batch = next(iter(test_dataset))
        module, loss, outputs, labels = eval_step(module, batch, batch_idx, training=False)

        test_batch_losses.append(loss.item())
        predictions.append(outputs)
        true_targets.append(labels)

    test_loss = np.mean(test_batch_losses)
#    module.test_log(test_batch_losses, test_loss)
    print(f"\nTest Loss : {test_loss}")
    return predictions, true_targets

In [ ]:
if __name__ == "__main__":
    EPOCHS = 1
    LR = 1e-4
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LR, eps=5e-8)
    module = train_loop(module=model,
                        EPOCHS=EPOCHS, 
                        train_dataset=train_dataloaded, 
                        val_dataset=val_dataloaded,
                        optimizer=optimizer)
    predictions, true_targets = evaluate(module, 
                                         test_dataloaded)



C:\Users\Mateo\AppData\Local\Temp\ipykernel_16816\2355751477.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key : torch.tensor(val[idx]) for key, val in self.encodings.items()}



Batch loss 1 : 10.690932273864746

Batch loss 2 : 7.545878887176514

Batch loss 3 : 5.737115859985352

Batch loss 4 : 5.110807418823242

Batch loss 5 : 4.460752487182617

Batch loss 6 : 4.176612377166748

Batch loss 7 : 3.3132517337799072

Batch loss 8 : 2.7446494102478027

Batch loss 9 : 2.3033745288848877

Batch loss 10 : 2.031712055206299

Batch loss 11 : 1.575617790222168

Batch loss 12 : 1.3664805889129639

Batch loss 13 : 1.1571484804153442

Batch loss 14 : 0.8873598575592041

Batch loss 15 : 0.7502452731132507

Batch loss 16 : 0.592736005783081

Batch loss 17 : 0.538517415523529

Batch loss 18 : 0.604697585105896

Batch loss 19 : 0.3212076425552368

Batch loss 20 : 0.4435454308986664

Batch loss 21 : 0.25785577297210693

Batch loss 22 : 0.26058661937713623

Batch loss 23 : 0.2633827030658722

Batch loss 24 : 0.25170719623565674

Batch loss 25 : 0.3286091983318329

Batch loss 26 : 0.20134860277175903

Batch loss 27 : 0.24497640132904053

Batch loss 28 : 0.15108948945999146

Batc

In [10]:
print(torch.cuda.memory_stats())

OrderedDict({'active.all.allocated': 2189543, 'active.all.current': 759, 'active.all.freed': 2188784, 'active.all.peak': 1024, 'active.large_pool.allocated': 1020003, 'active.large_pool.current': 277, 'active.large_pool.freed': 1019726, 'active.large_pool.peak': 381, 'active.small_pool.allocated': 1169540, 'active.small_pool.current': 482, 'active.small_pool.freed': 1169058, 'active.small_pool.peak': 721, 'active_bytes.all.allocated': 8415742742016, 'active_bytes.all.current': 10911057408, 'active_bytes.all.freed': 8404831684608, 'active_bytes.all.peak': 10917021184, 'active_bytes.large_pool.allocated': 8239346608128, 'active_bytes.large_pool.current': 10908502528, 'active_bytes.large_pool.freed': 8228438105600, 'active_bytes.large_pool.peak': 10913417728, 'active_bytes.small_pool.allocated': 176396133888, 'active_bytes.small_pool.current': 2554880, 'active_bytes.small_pool.freed': 176393579008, 'active_bytes.small_pool.peak': 13091328, 'allocated_bytes.all.allocated': 8415742742016, '

In [ ]:
print(inputs.input_ids.max())
print(inputs.input_ids.min())